In [ ]:
#if (!requireNamespace("BiocManager", quietly = TRUE))
#    install.packages("BiocManager")

#BiocManager::install("clusterProfiler", force = TRUE)
#BiocManager::install("DO.db", force=TRUE)

#install.packages("ontologyIndex")
#install.packages("ggnewscale")
#install.packages("ggupset")
#install.packages("europepmc")
#install.packages("ggridges")
#install.packages("forcats")
#install.packages("rentrez")

In [1]:
library(dplyr)
library(stringr)
library(tidyr)
library(clusterProfiler)
library(ontologyIndex)
library(ggnewscale)
library(enrichplot)
library(ggupset)
library(europepmc)
library(ggridges)
library(ggplot2)
library(cowplot)
library(forcats)
library(rentrez)

#load("../tea/A2TEA_finished.RData")


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




clusterProfiler v4.0.5  For help: https://yulab-smu.top/biomedical-knowledge-mining-book/

If you use clusterProfiler in published research, please cite:
T Wu, E Hu, S Xu, M Chen, P Guo, Z Dai, T Feng, L Zhou, W Tang, L Zhan, X Fu, S Liu, X Bo, and G Yu. clusterProfiler 4.0: A universal enrichment tool for interpreting omics data. The Innovation. 2021, 2(3):100141. doi: 10.1016/j.xinn.2021.100141


Attaching package: ‘clusterProfiler’


The following object is masked from ‘package:stats’:

    filter




### Building a Drought GO truth set
#### Using rentrez package to query NCBI

##### Idea 1: Based on investigated species

In [2]:
#get available NCBI database
entrez_dbs()

[1] "pubmed"          "protein"         "nuccore"         "ipg"            
 [5] "nucleotide"      "structure"       "genome"          "annotinfo"      
 [9] "assembly"        "bioproject"      "biosample"       "blastdbinfo"    
[13] "books"           "cdd"             "clinvar"         "gap"            
[17] "gapplus"         "grasp"           "dbvar"           "gene"           
[21] "gds"             "geoprofiles"     "homologene"      "medgen"         
[25] "mesh"            "ncbisearch"      "nlmcatalog"      "omim"           
[29] "orgtrack"        "pmc"             "popset"          "proteinclusters"
[33] "pcassay"         "protfam"         "biosystems"      "pccompound"     
[37] "pcsubstance"     "seqannot"        "snp"             "sra"            
[41] "taxonomy"        "biocollections"  "gtr"

In [ ]:
entrez_db_summary("annotinfo")

In [ ]:
entrez_db_summary("gene")

In [ ]:
drought_search <- entrez_search(db="gene", 
                                term="(drought[All Fields] OR desiccation[All Fields] OR water deprivation[All Fields]) AND alive[prop] AND (Arabidopsis[Organism] OR Hordeum[Organism] OR Zea mays[Organism])",
                                retmax=1000
                               )

head(drought_search$ids, n=40)
length(drought_search$ids)

In [5]:
entrez_summary(db="gene", id=816394)

,chrloc,chraccver,chrstart,chrstop,exoncount
,<chr>,<chr>,<int>,<int>,<int>
1,2,NC_003071.7,8139755,8144460,3


In [ ]:
aliases <- vector(length = length(drought_search$ids))

count = 1

for (q in drought_search$ids) {

  q_summary <- entrez_summary(db="gene", id=q)    
  aliases[count] <- q_summary$otheraliases
  #increment count, so next iteation goes to next index position
  count = count + 1
}

aliases

In [ ]:
aliases <- str_remove(aliases, ",(.*)")
aliases <- str_remove(aliases, "ZEAMMB73_")
aliases <- str_subset(aliases, "ARALY", negate = TRUE)
aliases <- str_subset(aliases, " ", negate = TRUE)

In [ ]:
aliases

In [ ]:
drought_GO_At <- SFA$Arabidopsis_thaliana %>%
  filter(`Protein-Accession` %in% aliases) %>%
  pull(`Gene-Ontology-Term`)

drought_GO_Zm <- SFA$Zea_mays %>%
  filter(`Protein-Accession` %in% aliases) %>%
  pull(`Gene-Ontology-Term`)

drought_GO_Hv <- SFA$Hordeum_vulgare %>%
  filter(`Protein-Accession` %in% aliases) %>%
  pull(`Gene-Ontology-Term`)

In [ ]:
m <- c(drought_GO_At, drought_GO_Zm, drought_GO_Hv)
n <- str_split(m, ",")
n <- unlist(n)
n <- na.omit(n)
n <- str_trim(n)
n <- unique(n)
length(n)
n

In [ ]:
ont_BP %>%
  filter(GO %in% n) %>%
  nrow()

ont_MF %>%
  filter(GO %in% n) %>%
  head(n=10)

In [ ]:
ont_MF %>%
  filter(GO %in% n) %>%
  nrow()

ont_MF %>%
  filter(GO %in% n) %>%
  head(n=10)

In [ ]:
length(drought_search$ids)

In [ ]:
myfxn <- function(idX, dbfromX, dbX){
  entrez_link(id=idX, dbfrom=dbfromX, db=dbX)
}

In [ ]:
#split into chunks because sometimes network errors etc..
#current_links_1_100 <- lapply(drought_search$ids[1:100], myfxn, dbfromX='gene', dbX='all')
#current_links_101_200 <- lapply(drought_search$ids[101:200], myfxn, dbfromX='gene', dbX='all')
#current_links_201_300 <- lapply(drought_search$ids[201:300], myfxn, dbfromX='gene', dbX='all')
#current_links_301_400 <- lapply(drought_search$ids[301:400], myfxn, dbfromX='gene', dbX='all')
#current_links_401_500 <- lapply(drought_search$ids[401:500], myfxn, dbfromX='gene', dbX='all')
#current_links_501_600 <- lapply(drought_search$ids[501:600], myfxn, dbfromX='gene', dbX='all')
#current_links_601_700 <- lapply(drought_search$ids[601:700], myfxn, dbfromX='gene', dbX='all')
#current_links_701_800 <- lapply(drought_search$ids[701:800], myfxn, dbfromX='gene', dbX='all')
#current_links_801_900 <- lapply(drought_search$ids[801:900], myfxn, dbfromX='gene', dbX='all')
#current_links_901_1000 <- lapply(drought_search$ids[901:1000], myfxn, dbfromX='gene', dbX='all')
#current_links_1001_1100 <- lapply(drought_search$ids[1001:1100], myfxn, dbfromX='gene', dbX='all')
current_links_1101_1200 <- lapply(drought_search$ids[1101:1200], myfxn, dbfromX='gene', dbX='all')
current_links_1201_1300 <- lapply(drought_search$ids[1201:1300], myfxn, dbfromX='gene', dbX='all')
current_links_1301_1400 <- lapply(drought_search$ids[1301:1400], myfxn, dbfromX='gene', dbX='all')
print("1 done")
current_links_1401_1500 <- lapply(drought_search$ids[1401:1500], myfxn, dbfromX='gene', dbX='all')
current_links_1501_1600 <- lapply(drought_search$ids[1501:1600], myfxn, dbfromX='gene', dbX='all')
current_links_1601_1700 <- lapply(drought_search$ids[1601:1700], myfxn, dbfromX='gene', dbX='all')
print("2 done")
current_links_1701_1800 <- lapply(drought_search$ids[1701:1800], myfxn, dbfromX='gene', dbX='all')
current_links_1801_1900 <- lapply(drought_search$ids[1801:1900], myfxn, dbfromX='gene', dbX='all')
current_links_1901_2000 <- lapply(drought_search$ids[1901:2000], myfxn, dbfromX='gene', dbX='all')
current_links_2001_2100 <- lapply(drought_search$ids[2001:2100], myfxn, dbfromX='gene', dbX='all')
print("3 done")
current_links_2101_2200 <- lapply(drought_search$ids[2101:2200], myfxn, dbfromX='gene', dbX='all')
current_links_2201_2300 <- lapply(drought_search$ids[2201:2300], myfxn, dbfromX='gene', dbX='all')
current_links_2301_2400 <- lapply(drought_search$ids[2301:2400], myfxn, dbfromX='gene', dbX='all')
print("4 done")
current_links_2401_2500 <- lapply(drought_search$ids[2401:2500], myfxn, dbfromX='gene', dbX='all')
current_links_2501_2600 <- lapply(drought_search$ids[2501:2600], myfxn, dbfromX='gene', dbX='all')
current_links_2601_2700 <- lapply(drought_search$ids[2601:2700], myfxn, dbfromX='gene', dbX='all')
current_links_2701_2800 <- lapply(drought_search$ids[2701:2800], myfxn, dbfromX='gene', dbX='all')
current_links_2801_2900 <- lapply(drought_search$ids[2801:2900], myfxn, dbfromX='gene', dbX='all')
current_links_2901_2906 <- lapply(drought_search$ids[2901:2906], myfxn, dbfromX='gene', dbX='all')


#current_links_201_300[[100]]$links$gene_protein

In [ ]:
all_current_links <- c(current_links_1_100,
  current_links_101_200,
  current_links_201_300,
  current_links_301_400,
  current_links_401_500,
  current_links_501_600,
  current_links_601_700,
  current_links_701_800,
  current_links_801_900,
  current_links_901_1000,
  current_links_1001_1100,
  current_links_1101_1200,
  current_links_1201_1300,
  current_links_1301_1400,
  current_links_1401_1500,
  current_links_1501_1600,
  current_links_1601_1700,
  current_links_1701_1800,
  current_links_1801_1900,
  current_links_1901_2000,
  current_links_2001_2100,
  current_links_2101_2200,
  current_links_2201_2300,
  current_links_2301_2400,
  current_links_2401_2500,
  current_links_2501_2600,
  current_links_2601_2700,
  current_links_2701_2800, 
  current_links_2801_2900,
  current_links_2901_2906)

In [ ]:
length(all_current_links)

all_protein_ids <- vector() #list(length(current_links))

for (i in 1:length(all_current_links)) {
    all_protein_ids <- c(all_protein_ids, all_current_links[[i]]$links$gene_protein)
}

length(all_protein_ids)
all_protein_ids

In [ ]:
cat(file = "aaaa",
    strwrap(
      substr(entrez_fetch(db="protein", id="1789589066", rettype="fasta"), 
             1, 
             1e9)
    ), sep="\n", append = TRUE
  )

In [ ]:
#entrez_summary(db="protein", id=1063705582, rettype="fasta")
a <- entrez_summary(db="protein", id=current_links[[1]]$links$gene_protein, rettype="fasta")



b <- rep(NA, length(a))
#b

c <- lapply(a, "[", "slen")

for (i in 1:length(a)){
    b[i] <- c[[i]][[1]]
}

#neat - if multiple maxima returns the first ;D
d <- which.max(b)

current_links[[1]]$links$gene_protein[d]

In [ ]:
t <- entrez_fetch(db="protein", id=1063705582, rettype="fasta")
#t

#install.packages("seqinr")
library(seqinr)

f <- read.fasta(file = textConnection(t), as.string = T, whole.header = TRUE, seqtype = "AA")
f

write.fasta(sequences = f, names = names(f), file.out = "all_proteins_drought.fasta", open = "a", nbchar = 60, as.string = TRUE)

In [ ]:
names(f)

In [ ]:
#t <- entrez_fetch(db="protein", id=1789589066, rettype="fasta")

myfun <- function(idX){
    ef <- entrez_fetch(db="protein", id=idX, rettype="fasta")
    f <- read.fasta(file = textConnection(ef), as.string = T, whole.header = TRUE, seqtype = "AA")
    write.fasta(sequences = f, 
                names = names(f), 
                file.out = "all_proteins_drought.fasta", 
                open = "a", 
                nbchar = 60, 
                as.string = TRUE)
}

#lapply(c("1789589066", "1789589066"), myfun)
lapply(all_protein_ids, myfun)

In [ ]:
head(all_proteins_fasta)

In [ ]:
#### not shown - AHRD based GO term annotation of all proteins
#### next read-in of these annotation

In [ ]:
drought_AHRD <- read.delim("../../../ML-drought/AHRD_Snakemake/results/drought_all_flowering.ahrd_output.tsv", sep="\t")

In [ ]:
drought_AHRD <- drought_AHRD %>% 
    select(`Gene.Ontology.Term`) %>%
    #remove redundancy while collapsing with ", "
    summarise(`Gene.Ontology.Term` = paste(unique(`Gene.Ontology.Term`), collapse=", ")) %>%
    #we should remove all rows that have NO GO terms associated
    #and remove all NAs from the Gene-Ontology-Term column if they have at least 1 GO term
    #remove inline NAs in Gene-Ontology-Term column
    mutate(
      `Gene.Ontology.Term` = str_remove_all(`Gene.Ontology.Term`, "NA, |, NA") 
    ) %>% 
    #remove lines with only NA in Gene-Ontology-Term column
    filter(!str_detect(`Gene.Ontology.Term`, 'NA')) %>% 
    #get column as vector
    pull() %>% 
    #split into individual vector elements
    str_split(., pattern = ", ") %>%
    unique() %>%
    unlist()

In [ ]:
length(drought_AHRD)
head(drought_AHRD)

In [ ]:
BP_drought_set <- ont_BP %>%
  filter(GO %in% drought_AHRD) %>%
  pull(GO)

MF_drought_set <- ont_MF %>%
  filter(GO %in% drought_AHRD) %>%
  pull(GO) 

length(BP_drought_set)
length(MF_drought_set)

In [ ]:
drought_GO_BP_GTS

In [ ]:
intersect(BP_drought_set, drought_GO_BP_GTS)

In [ ]:
head(BP_drought_set)

## Build a complete embryophyta set - as ground truth

In [6]:
#05.11.2021 - 6308230 exist
#set sci notation to numbers - rentrez package has some weird errors otherwise
#1e13 
options("scipen"=100)
#embryophyta_search_05_11_2021 <- entrez_search(db="gene", 
#                                term="Embryophyta[Organism] AND alive[prop]",
#                                retmax=#6308230
#                   )

embryophyta_search_05_11_2021 <- readRDS("/scratch2/cropbio/stoecker/A2TEA-pipeline/ML-drought/rentrez_searches/embryophyta_search_05_11_2021")

length(embryophyta_search_05_11_2021$ids)
embryophyta_search_05_11_2021$ids[1]

[1] 6308230

[1] "816394"

In [286]:
str(embryophyta_search_05_11_2021$ids)

 chr [1:6308230] "816394" "830878" "817857" "830996" "842859" "842733" ...


In [296]:
#function to extract protein references from gene entry
#100 exectuons ~77-140s =~100s
#so roughly a one search one execution relationship
#200.000s/3600=~56h
#myfxn <- function(idX, dbfromX, dbX){
#  entrez_link(id=idX, dbfrom=dbfromX, db=dbX)$links$gene_protein
#}

myfxn <- function(idX, dbfromX, dbX){
  entrez_link(id=idX, dbfrom=dbfromX, db=dbX)$links$gene_protein
}

In [284]:
#t <- entrez_link(id=113299905, dbfrom='gene', db='protein')
t$links

elink result with information from 2 databases:
[1] gene_protein        gene_protein_refseq

In [234]:
#sampling randomly - add 50% to make sure to have genes with protein records (25% expected to be unusable)
#genes without protein records are NULL elements in vector and lead to downstream problems
#we circumvent by sampling more and filtering for non NULL
sample_test10 <- sample(embryophyta_search_05_11_2021$ids, 15, replace=FALSE)
sample_02k <- sample(embryophyta_search_05_11_2021$ids, 300, replace=FALSE)
sample_2k <- sample(embryophyta_search_05_11_2021$ids, 3000, replace=FALSE)
sample_20k <- sample(embryophyta_search_05_11_2021$ids, 30000, replace=FALSE)
sample_200k <- sample(embryophyta_search_05_11_2021$ids, 300000, replace=FALSE)
sample_2000k <- sample(embryophyta_search_05_11_2021$ids, 3000000, replace=FALSE)

In [283]:
str(sample_2000k)

 chr [1:3000000] "107648132" "106770945" "109738882" "120676655" "18600251" ...


In [235]:
#writing to current_links
system.time(
assign(paste0("current_links_", "10"), lapply(sample_test10, myfxn, dbfromX='gene', dbX='protein'))
    )

   user  system elapsed 
  0.331   0.019  11.257 

In [282]:
#now in loop so if connection times out etc. we have all prev. results saved at least..
for (i in seq(1,10, by = 5)){
    assign(paste0("new_current_links_", i), lapply(sample_test10[i:(i+4)], myfxn, dbfromX='gene', dbX='protein'))
    print(i)
    print(paste("current_links number", i, "created succesfully! ;D"))
}

[1] 1
[1] "current_links number 1 created succesfully! ;D"
[1] 6
[1] "current_links number 6 created succesfully! ;D"


In [281]:
assign(paste0("new_current_links_", i), lapply(sample_test10[1:(6+4)], myfxn, dbfromX='gene', dbX='protein'))
#new_current_links_6 <- lapply(sample_test10[1:(6+4)], myfxn, dbfromX='gene', dbX='protein')
str(new_current_links_6)

List of 10
 $ : NULL
 $ : chr "1954721948"
 $ : chr "694377018"
 $ : NULL
 $ : chr [1:2] "1904803248" "1878255501"
 $ : NULL
 $ : chr "1173147887"
 $ : chr "1012007324"
 $ : chr "1040883037"
 $ : chr "1585753397"


In [280]:
sample_test10
tt <- lapply(sample_test10[1:(6+4)], myfxn, dbfromX='gene', dbX='protein')
str(tt)

[1] "107936760" "120007389" "103954737" "25017151"  "58903201"  "107494924"
 [7] "110006571" "107470398" "108221744" "114274584" "110787726" "103995147"
[13] "107468368" "101222780" "110035832"

List of 10
 $ : NULL
 $ : chr "1954721948"
 $ : chr "694377018"
 $ : NULL
 $ : chr [1:2] "1904803248" "1878255501"
 $ : NULL
 $ : chr "1173147887"
 $ : chr "1012007324"
 $ : chr "1040883037"
 $ : chr "1585753397"


In [202]:
length(
current_links[lengths(current_links) != 0]
)

#current_links[[2]]$links$gene_protein

[1] 12

In [205]:
current_links <- current_links[lengths(current_links) != 0][1:10]
str(current_links)

List of 10
 $ : chr [1:3] "2070073149" "2070073147" "2043533359"
 $ : chr "1147559588"
 $ : chr [1:2] "1269946173" "1269946171"
 $ : chr "1731673874"
 $ : chr "698514890"
 $ : chr [1:2] "2060419465" "2041630615"
 $ : chr "727599467"
 $ : chr "1025221250"
 $ : chr [1:2] "944064987" "357136825"
 $ : chr "1117373241"


In [237]:
#a <- 
entrez_summary(db="protein", id=current_links[[2]], rettype="fasta")
#length(current_links[[2]])

,type,count,subtype,source
,<chr>,<int>,<chr>,<chr>
1,all,4,NA,NA
2,blob_size,4202,NA,NA
3,cdregion,1,NA,NA
4,cdregion,1,CDS,NA
5,gene,1,NA,NA
6,gene,1,Gene,NA
7,org,1,NA,NA
8,prot,1,NA,NA
9,prot,1,Prot,NA


In [221]:
#entrez_summary(db="protein", id=1063705582, rettype="fasta")
#a <- entrez_summary(db="protein", id=current_links[[1]]$links$gene_protein, rettype="fasta")
a <- entrez_summary(db="protein", id=current_links[[2]], rettype="fasta")
length(a)
#a


b <- rep(NA, length(a))
b

c <- lapply(a, "[", "slen")
c
#c <- lapply(a, "slen")

#for (i in 1:length(a)){
#    b[i] <- c[[i]][[1]]
#}

#neat - if multiple maxima returns the first ;D
#d <- which.max(b)

#current_links[[1]]$links$gene_protein[d]
#current_links[[1]][d]

[1] 32

[1] NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA
[26] NA NA NA NA NA NA NA

ERROR: Error in `[.data.frame`(X[[i]], ...): undefined columns selected


In [232]:
#as a loop retruning a vactor of all longest proteins/isoforms for each gene in input

final_size = rep(NA, length(current_links))


for (links in 1:length(current_links)) {
#    print(links)
    a <- entrez_summary(db="protein", id=current_links[[links]], rettype="fasta")
#    print(a)

   #if more than element do this - else just return
   #important to check the current_links element and not a - both are lists!
    if (length(current_links[[links]]) > 1) {
               
        b <- rep(NA, length(a)) 
        c <- lapply(a, "[", "slen")
    
        for (i in 1:length(a)){
            b[i] <- c[[i]][[1]]
        }

        #neat - if multiple maxima returns the first ;D
        d <- which.max(b)
        
    } else {
        d <- 1
    }

    print(current_links[[links]][d])
}







[1] "2043533359"
[1] "1147559588"
[1] "1269946171"
[1] "1731673874"
[1] "698514890"
[1] "2060419465"
[1] "727599467"
[1] "1025221250"
[1] "944064987"
[1] "1117373241"


In [287]:
load("/scratch2/cropbio/stoecker/A2TEA-pipeline/ML-drought/rentrez_searches/300k.RDATA")

In [301]:
str(sample_02k)

 chr [1:300] "18792006" "21398409" "112511794" "30511508" "103984935" ...


In [302]:
for (i in seq(1,300, by = 10)){
  assign(paste0("new2_current_links_", i), lapply(sample_02k[i:(i+9)], myfxn, dbfromX='gene', dbX='protein'))
  print(paste(i+9, "of 300 link sets created succesfully! ;D"))
}

[1] "10 of 30 link sets created succesfully! ;D"
[1] "20 of 30 link sets created succesfully! ;D"
[1] "30 of 30 link sets created succesfully! ;D"
[1] "40 of 30 link sets created succesfully! ;D"
[1] "50 of 30 link sets created succesfully! ;D"
[1] "60 of 30 link sets created succesfully! ;D"
[1] "70 of 30 link sets created succesfully! ;D"
[1] "80 of 30 link sets created succesfully! ;D"
[1] "90 of 30 link sets created succesfully! ;D"
[1] "100 of 30 link sets created succesfully! ;D"
[1] "110 of 30 link sets created succesfully! ;D"
[1] "120 of 30 link sets created succesfully! ;D"
[1] "130 of 30 link sets created succesfully! ;D"
[1] "140 of 30 link sets created succesfully! ;D"
[1] "150 of 30 link sets created succesfully! ;D"
[1] "160 of 30 link sets created succesfully! ;D"
[1] "170 of 30 link sets created succesfully! ;D"
[1] "180 of 30 link sets created succesfully! ;D"
[1] "190 of 30 link sets created succesfully! ;D"
[1] "200 of 30 link sets created succesfully! ;D"
[1] "210 

In [323]:
new2_list <- ls(pattern = "new2_current_links_")
str(new2_list)

ccc = list()
for (i in new2_list) {
    ccc <- c(ccc, get(i))
}

str(ccc)
unlist(ccc)

 chr [1:30] "new2_current_links_1" "new2_current_links_101" ...
List of 300
 $ : chr "1162512518"
 $ : chr [1:2] "703121559" "587905127"
 $ : chr "1389817043"
 $ : NULL
 $ : chr [1:2] "1091711313" "695026946"
 $ : NULL
 $ : chr [1:2] "1150718135" "1141984789"
 $ : chr "1104638758"
 $ : chr [1:7] "1444408161" "1444408159" "1444408156" "1444408154" ...
 $ : chr [1:3] "242061798" "241932019" "189345346"
 $ : NULL
 $ : chr "225449004"
 $ : chr [1:2] "1833213198" "1833213196"
 $ : NULL
 $ : chr [1:3] "1834966225" "1825511409" "1825511337"
 $ : chr "1130643969"
 $ : chr "1757590032"
 $ : chr [1:2] "1431729556" "1431729554"
 $ : chr "1150105785"
 $ : chr [1:8] "1932771132" "1173846781" "1173846779" "1173846777" ...
 $ : chr [1:2] "1219069750" "1219069748"
 $ : chr [1:6] "1780604938" "332194302" "75215676" "26449682" ...
 $ : chr [1:2] "1942099144" "1887140962"
 $ : NULL
 $ : chr [1:3] "823226181" "823226179" "763790215"
 $ : chr [1:2] "1350302905" "1350302900"
 $ : chr [1:2] "1573132485" "155

[1] "1162512518" "703121559"  "587905127"  "1389817043" "1091711313"
  [6] "695026946"  "1150718135" "1141984789" "1104638758" "1444408161"
 [11] "1444408159" "1444408156" "1444408154" "1444408152" "1444408150"
 [16] "1104610279" "242061798"  "241932019"  "189345346"  "225449004" 
 [21] "1833213198" "1833213196" "1834966225" "1825511409" "1825511337"
 [26] "1130643969" "1757590032" "1431729556" "1431729554" "1150105785"
 [31] "1932771132" "1173846781" "1173846779" "1173846777" "1173846775"
 [36] "1173846773" "1173846771" "643710465"  "1219069750" "1219069748"
 [41] "1780604938" "332194302"  "75215676"   "26449682"   "18402618"  
 [46] "5430762"    "1942099144" "1887140962" "823226181"  "823226179" 
 [51] "763790215"  "1350302905" "1350302900" "1573132485" "1552056665"
 [56] "1389750745" "1389750741" "1999119369" "1914212561" "1973698322"
 [61] "1973698320" "1973698317" "629089545"  "629089544"  "698424019" 
 [66] "1573246346" "1484744393" "1484744391" "1484744389" "1484744387"
 [71] "1072926838" "1471967973" "922333128"  "657372793"  "1220045273"
 [76] "332646255"  "75264601"   "15231020"   "6735351"    "743900910" 
 [81] "743900908"  "743900906"  "1540558212" "1968367758" "1333135877"
 [86] "1279090569" "1590610549" "1572866369" "1662935123" "1012337161"
 [91] "1012337160" "1531816884" "2029730392" "2029730390" "2029730387"
 [96] "1994294476" "1994294475" "1028978079" "1028978077" "1117402605"
[101] "1102699529" "1025088501" "2055415414" "1945587390" "1931481151"
[106] "502114054"  "502114051"  "1073026158" "2123639621" "1531793804"
[111] "1002276309" "937921187"  "215701067"  "5803264"    "817527161" 
[116] "808178154"  "1847691710" "1824635161" "1026003183" "1915405753"
[121] "1894457556" "1044530485" "920682414"  "1531825352" "1531825350"
[126] "1531825348" "1531825345" "1531825343" "1531825341" "1531825338"
[131] "1531825335" "1531825331" "1531825329" "1531825327" "1531825325"
[136] "1531825323" "1531825321" "1931498692" "1662932363" "700204688" 
[141] "449458173"  "1658164097" "729391492"  "729391489"  "1142624358"
[146] "1142624357" "1142624356" "1121393426" "257333204"  "223974997" 
[151] "219746390"  "194701722"  "11385513"   "1109301976" "970010784" 
[156] "2123801160" "2123801157" "1079218020" "848902249"  "604311826" 
[161] "1040828614" "1021054340" "1729942513" "1511245881" "1478168369"
[166] "1281005269" "2088250753" "1959385197" "1973713884" "700253426" 
[171] "629082641"  "629082640"  "1999480028" "1981822323" "1269885227"
[176] "1269885225" "1333162761" "1333162759" "1333162757" "1333162755"
[181] "1139791360" "1139791359" "596004271"  "383386109"  "383386107" 
[186] "383386105"  "383386103"  "383386101"  "383386099"  "383386097" 
[191] "383386095"  "383386093"  "383386091"  "374671153"  "11967889"  
[196] "1035397180" "659081907"  "659081905"  "697145110"  "1175891677"
[201] "568868517"  "2123992622" "1931567064" "1931567062" "1279759288"
[206] "1369121931" "1343893725" "1833234413" "697096310"  "697096308" 
[211] "697096306"  "1931458142" "1369119679" "1167934531" "1965190963"
[216] "1776560694" "1219140864" "1219140862" "1999113711" "1900537958"
[221] "923887967"  "160693712"  "727596330"  "727596328"  "727596326" 
[226] "1175844212" "1431740828" "41072339"   "1484667276" "1624092178"
[231] "1464315057" "1441553013" "1333113395" "970068501"  "1610628211"
[236] "1590111509" "698587034"  "2029524952" "1882635785" "1865393517"
[241] "1865393516" "1865393515" "1098750469" "1098750467" "1098750465"
[246] "848895702"  "604316166"  "698568048"  "2123654136" "672159206" 
[251] "1226780633" "1968399149" "1729953169" "1729953167" "1729953165"
[256] "1729953163" "1279763014" "1919573725" "1909390864" "1994393223"
[261] "1994393222" "1028967234" "1028967232" "1540564324" "2088243928"
[266] "1953564685" "1953564662" "1666294761" "1666294759" "1666294757"
[271] "1666294755" "1666294753" "1666294751" "1666294748" "2060432380"
[276] "2041627759" "2069928663" "2043580752" "2060324892" "2041661445"
[281] "1150748998" "985473

In [311]:
c(get(paste0("new2_current_links_", "1")), get(paste0("new2_current_links_", "2"))) 

ERROR: Error in get(paste0("new2_current_links_", "2")): object 'new2_current_links_2' not found


In [325]:
list300k <- ls(pattern = "new_current_links_")
str(list300k)

ccc = list()
for (i in list300k) {
    ccc <- c(ccc, get(i))
}

str(ccc)
#length(unlist(ccc))

 chr [1:300] "new_current_links_1" "new_current_links_100001" ...
List of 299005
 $ : chr [1:2] "1376897150" "300171498"
 $ : chr "2082418651"
 $ : chr [1:4] "441018047" "29839379" "18860341" "18389494"
 $ : chr "1012169493"
 $ : chr "1142634124"
 $ : chr "1527495545"
 $ : chr "224035231"
 $ : chr "1964483091"
 $ : chr [1:2] "1040863937" "1021042252"
 $ : chr "2123933186"
 $ : chr [1:3] "1860341288" "1860341286" "1635318519"
 $ : chr "1035408889"
 $ : chr "1729928570"
 $ : chr "2022339264"
 $ : chr [1:7] "1142676838" "1142676837" "1142676836" "1142676835" ...
 $ : chr [1:3] "1376990859" "1376990857" "300138087"
 $ : chr [1:2] "722491685" "704002280"
 $ : chr [1:3] "1040852691" "1040852688" "1021043812"
 $ : chr [1:3] "1040842599" "1040842597" "1021048137"
 $ : chr [1:3] "1111120801" "1111120797" "1102144287"
 $ : chr [1:2] "1350188167" "587933605"
 $ : chr "1431584120"
 $ : chr [1:3] "1269910020" "1269910018" "1269910016"
 $ : chr "1350327027"
 $ : NULL
 $ : NULL
 $ : chr "2097428998"


[1] 471312